In [1]:
import numpy as np
import sys
import os
import pickle
import glob
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from DEFAULTS import PARENT_PATH
import sim.ap_field_ar as ap
import sim.far_field_mp as ff
import sim.tele_geo_ar as tg
import sim.pan_mod_ar as pm

In [2]:
# load simulation data 
# sim_file = "E:\\Holography\\lat_holocal\\data\\ctcheung\\2022_02_04_04_05\\simdata_2.pys"
sim_file = "E:\\Holography\\lat_holocal\\data\\ctcheung\\focal_field_35.pys"
# dataset = dict(adj_err_1=[], adj_err_2=[], field_fp=[])
dataset = {}
with open(sim_file, "rb") as file:
    sim_data = pickle.load(file)
dataset["adj_err_1"] = sim_data["adj_err_1"]
dataset["adj_err_2"] = sim_data["adj_err_2"]
dataset["field_fp"] = sim_data["field_fp"]
print(len(dataset["field_fp"]))

100


In [3]:
# create panel offsets randomly
save = 0 
adj_1_A = dataset["adj_err_1"]
adj_2_A = dataset["adj_err_2"]
# error_rms = 0
# adj_1_A = np.random.randn(1092) * error_rms
# adj_2_A = np.random.randn(1092) * error_rms
panel_model2 = pm.panel_model_from_adjuster_offsets(
    2, adj_2_A, 1, save
)  # Panel Model on M2
panel_model1 = pm.panel_model_from_adjuster_offsets(
    1, adj_1_A, 1, save
)  # Panel Model on M1

In [12]:
# ###################### define telescope geometry
freq = 101.28 # frequency of signal source [GHz]
wavelength = (30.0 / freq) * 0.01 # [m]
wavelength = wavelength * 1e3 # [mm]
k = 2 * np.pi / wavelength
P_source = [0, -7200.0, 1e6] # unit [mm]
st_th_fwhp = 30.0/180.0*np.pi 

tele_geo = tg.TelescopeGeometry()
tele_geo.N_scan = 50  # pixels in 1D of grid
# tele_geo.de_ang = 0.2 / 60 * np.pi / 180  # angle increment of telescope scan [rad]
arcmin_to_rad = 1.0 / 60 * np.pi / 180.0
tele_geo.de_ang = 60.0/tele_geo.N_scan / (252.0/60.0) * arcmin_to_rad   # angle increment of telescope scan [rad]

tele_geo.lambda_ = wavelength / 1e3 # [m]
tele_geo.k = 2 * np.pi / tele_geo.lambda_  # wavenumber [1/m]
# tele_geo.th_fwhp = 44 * np.pi / 180  # full width half power of source feed [rad]
tele_geo.z_tow = 1e3 #[m]
# Azimuth and Elevation center [rad]
tele_geo.az0 = 0.0
tele_geo.el0 = np.arctan(-tele_geo.y_tow / tele_geo.z_tow)
# el0 = 0.005
# position of the receiver feed [mm]
P_rx = np.array([0, 209.920654, 0])
# P_rx = np.array([0, 0, 0])
[tele_geo.rx_x, tele_geo.rx_y, tele_geo.rx_z] = P_rx/1e3

# arrays of pointing angles of rays
angle_size = 0.29
theta_a, theta_b, theta_N = -np.pi / 2 - angle_size, -np.pi / 2 + angle_size, 100
phi_a, phi_b, phi_N = np.pi / 2 - angle_size, np.pi / 2 + angle_size, 100
theta = np.linspace(theta_a, theta_b, theta_N)
phi = np.linspace(phi_a, phi_b, phi_N)



# get parameters from telescope geometry
tg_th_1, tg_th2, tg_z_ap = tele_geo.th_1, tele_geo.th2, tele_geo.z_ap
tg_th_fwhp, tg_F_2 = tele_geo.th_fwhp, tele_geo.F_2

# ray tracing
rxmirror = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
apout = ap.aperature_fields_from_panel_model(panel_model1, panel_model2, \
                                    P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                    theta, phi, rxmirror
                                    )

In [13]:
# import numba
# # @numba.jit(nopython=True, parallel=True, fastmath=True)
# def far_field_sim(ap_field, msmt_geo):
#     # Break out many quantities from msmt_geo
#     N_scan = msmt_geo.N_scan
#     de_ang = msmt_geo.de_ang
#     lambda_ = msmt_geo.lambda_

#     x_tow = msmt_geo.x_tow
#     y_tow = msmt_geo.y_tow
#     z_tow = msmt_geo.z_tow

#     x_phref = msmt_geo.x_phref
#     y_phref = msmt_geo.y_phref
#     z_phref = msmt_geo.z_phref

#     x_rotc = msmt_geo.x_rotc
#     y_rotc = msmt_geo.y_rotc
#     z_rotc = msmt_geo.z_rotc

#     el0 = msmt_geo.el0
#     az0 = msmt_geo.az0

#     # Break out the geometric coordinates from ap_fields
#     # Location of points on the aperture plane
#     # in rotation centered coordinates
#     x_ap = ap_field[9, :] / 1e3 - x_rotc
#     y_ap = ap_field[10, :] / 1e3 - y_rotc
#     z_ap = ap_field[11, :] / 1e3 - z_rotc
#     N_apscan = len(x_ap)
#     # x_ap_tile = np.reshape(np.tile(x_ap, N_scan), (N_scan,len(x_ap)))
#     # y_ap_tile = np.reshape(np.tile(y_ap, N_scan), (N_scan,len(x_ap)))
#     # z_ap_tile = np.reshape(np.tile(z_ap, N_scan), (N_scan,len(x_ap)))

#     # Propagation vector of the sample points (tan_og)
#     k_x = ap_field[12, :]
#     k_y = ap_field[13, :]
#     k_z = ap_field[14, :]

#     pathl = ap_field[15, :] / 1e3  # Path length convert to meters
#     ampl = np.sqrt(ap_field[16, :])  # Amplitude

#     k = 2.0 * np.pi / lambda_  # Wavenumber [1/m]

#     # az, el angles for rotation
#     el_cur = np.linspace(el0-N_scan*de_ang, el0+N_scan*de_ang, 2*N_scan)
#     az_cur = np.linspace(az0-N_scan*de_ang, az0+N_scan*de_ang, 2*N_scan)
#     el_cur, az_cur = np.meshgrid(el_cur, az_cur)
#     el_cur = np.ravel(el_cur)
#     az_cur = np.ravel(az_cur)
    
#     # Complex fields
#     ima = np.complex(0, 1)
#     Fcomplex = ampl * np.exp(ima * pathl * k)

#     Npts = len(x_ap)
#     out = np.zeros((3, N_scan * N_scan * 4), dtype=complex)
#     oneselcur = np.ones(len(el_cur))

#     # Elevation rotation (about x axis)
#     sinelcur = np.sin(el_cur)
#     coselcur = np.cos(el_cur)
#     x_temp = np.outer(oneselcur, x_ap)
#     y_temp = np.outer(coselcur, y_ap) - np.outer(sinelcur, z_ap)
#     z_temp = np.outer(sinelcur, y_ap) + np.outer(coselcur, z_ap)

#     cosazmatrx = np.outer(np.cos(az_cur), np.ones(N_apscan)) 
#     sinazmatrx = np.outer(np.sin(az_cur), np.ones(N_apscan)) 
#     x_apr = cosazmatrx * x_temp + sinazmatrx * z_temp
#     y_apr = y_temp
#     z_apr = -sinazmatrx * x_temp + cosazmatrx * z_temp

#     # Evaluate the distance to the phase reference if prompted to do so

#     x_temp = x_phref
#     y_temp = np.cos(el_cur) * y_phref - np.sin(el_cur) * z_phref
#     z_temp = np.sin(el_cur) * y_phref + np.cos(el_cur) * z_phref

#     x_phrefr = np.cos(az_cur) * x_temp + np.sin(az_cur) * z_temp
#     y_phrefr = y_temp
#     z_phrefr = -np.sin(az_cur) * x_temp + np.cos(az_cur) * z_temp

#     r_phref = np.sqrt(
#         (x_phrefr - x_tow) * (x_phrefr - x_tow)
#         + (y_phrefr - y_tow) * (y_phrefr - y_tow) 
#         + (z_phrefr - z_tow) * (z_phrefr - z_tow)
#     )

#     # Evaluate r
#     r = np.sqrt((x_apr - x_tow)*(x_apr - x_tow) + (y_apr - y_tow)*(y_apr - y_tow)+ (z_apr - z_tow)*(z_apr - z_tow))
#     z_dot_rhat = (z_apr - z_tow) * (-1) / r

#     out[0] = az_cur
#     out[1] = el_cur
#     out[2] = (
#                 np.exp(-ima * r_phref * k)
#                 * np.sum(
#                     (Fcomplex * np.exp(ima * k * r) / (4 * np.pi * r))
#                     * ((ima * k + 1 / r) * z_dot_rhat + ima * k * np.outer(oneselcur, k_z)), axis=1)
#                 / Npts
#             )

#     return out

farbeam = ff.far_field_sim(apout, tele_geo, None)

In [14]:
# ################################# plot the graph #########################################
# x_C = farbeam[0].real
# y_C = farbeam[1].real
# beam = farbeam[2]
# amp = np.abs(beam)
# c_C = 20*np.log10(amp/np.max(amp))

# c_C = np.angle(beam)
# c_C = np.mod(c_C+ 2.1+np.pi, np.pi*2)
# # c_C = c_C-c_C[int(len(c_C)/2.0)]

# # c_C = np.flip(c_C, axis=0).T 
# import matplotlib.pyplot as plt
# # plt.rcParams["image.cmap"] = "magma"
# plt.rcParams["image.cmap"] = "twilight"
# plt.figure(figsize=(5, 3.5))
# plt.title("", fontsize=20, x=0.44, y=1.15)
# plt.scatter(x_C, y_C, c=c_C)
# plt.axis("equal")
# plt.xlabel("x []")
# plt.ylabel("y []")
# plt.colorbar(label=r"$ dB $")
# plt.show()

In [15]:
# imag = np.reshape(c_C, (2*tele_geo.N_scan, 2*tele_geo.N_scan))
# imag = np.rot90(np.flip(imag), 2, (0,1)) 
# plt.imshow(imag)

In [16]:
nx = len(farbeam[0])
ny = len(farbeam[1])
ndat = len(farbeam[0])
azi = farbeam[0].real
az = azi
eli = farbeam[1].real
el = eli
beam = farbeam[2] 
nscan = int(np.sqrt(len(el)))
AZ = np.reshape(az, (nscan, nscan))
EL = np.reshape(el, (nscan, nscan))
beam = np.reshape(beam, (nscan, nscan))

beammain = beam[int(nscan/2), int(nscan/2)]
beam_sky = beam * np.conj(beammain)
beam_sky = beam_sky / np.max(np.abs(beam_sky)) # normalize the field
beam_sky =  np.rot90(beam_sky.T, 1, (0, 1))

In [17]:
############# plot the field #######################################
import plotly.graph_objects as go
y = AZ[:,0]*1e3
x = EL[0,:]*1e3
field = beam_sky
field = field / np.max(np.abs(field)) # normalize the field
# field = np.rot90(np.flip(field), 2, (0,1)) 
# field = np.rot90(np.flip(field), 3, (0,1)) 
# field = np.rot90(field.T, 1, (0, 1))
phase = np.angle(field)
phase = np.mod(phase, 2*np.pi)
# phase = np.mod(phase+ 2.1+np.pi, np.pi*2)
# amp = np.abs(field)
amp = 20*np.log10(np.abs(field))

title = 'Beam on sky'
plot1title = "Phase [rad]"
plot2title = "Intensity [dB]"
xlabel = "el [mrad]"
ylabel = "az [mrad]"
plot_1 = go.Heatmap(
                    x=x, 
                    y=y,
                    z = phase,
                    colorscale='Twilight',
                    showscale=True, colorbar=dict(len=0.8, x=0.44),
                    showlegend = False,
                    # hoverinfo='name', 
                    name="phase"
                    )
plot_2 = go.Heatmap( 
                    x=x, 
                    y=y,
                    z = amp,
                    # z= np.abs(field_fp*field_fp),
                    colorscale='Magma',
                    showscale=True, 
                    colorbar=dict(len=0.8, x=1), 
                    showlegend = False,
                    # hoverinfo='name', 
                    name="power"
                    )
layout = go.Layout(title=title, autosize=True,
                width=1000, height=500, 
                margin=dict(l=50, r=50, b=65, t=90),
                xaxis1 = dict(title=xlabel),
                yaxis1 = dict(scaleanchor = 'x', title=ylabel),
                xaxis2 = dict(scaleanchor = "x", title=xlabel),
                yaxis2 = dict(scaleanchor = "y"),
                )

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, shared_xaxes=True, subplot_titles=[plot1title, plot2title])
fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)
fig.update_layout(layout)
fig.show()

In [18]:
# load simulation data 
Npts_x = np.shape(dataset["field_fp"])[0]
Npts_z = np.shape(dataset["field_fp"])[1]
scanrange_x = 120 # [mm]
scanrange_z = 120 # [mm]
P_rx_x = np.linspace(-scanrange_x/2.0, scanrange_x/2.0, Npts_x)
P_rx_y = 209.920654 # mm, position of focus point with rf source located 1km away
P_rx_z = np.linspace(-scanrange_z/2.0, scanrange_z/2.0, Npts_z)

beam_fp = dataset["field_fp"]
beammain = beam_fp[int(Npts_x/2), int(Npts_z/2)]
beam_fp = beam_fp * np.conj(beammain)
beam_fp = beam_fp / np.max(np.abs(beam_fp))
############# plot the field #######################################
import plotly.graph_objects as go
y = P_rx_x
x = P_rx_z
field = beam_fp
field = field / np.max(np.abs(field)) # normalize the field
# field = np.rot90(np.flip(field), 2, (0,1)) 
# field = np.rot90(np.flip(field), 3, (0,1)) 
phase = np.angle(field)
phase = np.mod(phase, 2*np.pi)
# phase = np.mod(phase+ 2.1+np.pi, np.pi*2)
amp = 20*np.log10(np.abs(field))

title = 'Beam on focal plane'
plot1title = "Phase [rad]"
plot2title = "Intensity [dB]"
xlabel = "x [mm]"
ylabel = "y [mm]"
plot_1 = go.Heatmap(
                    x=x, 
                    y=y,
                    z = phase,
                    colorscale='Twilight',
                    showscale=True, colorbar=dict(len=0.8, x=0.44),
                    showlegend = False,
                    # hoverinfo='name', 
                    name="phase"
                    )
plot_2 = go.Heatmap( 
                    x=x, 
                    y=y,
                    z = amp,
                    # z= np.abs(field_fp*field_fp),
                    colorscale='Magma',
                    showscale=True, 
                    colorbar=dict(len=0.8, x=1), 
                    showlegend = False,
                    # hoverinfo='name', 
                    name="power"
                    )
layout = go.Layout(title=title, autosize=True,
                width=1000, height=500, 
                margin=dict(l=50, r=50, b=65, t=90),
                xaxis1 = dict(title=xlabel),
                yaxis1 = dict(scaleanchor = 'x', title=ylabel),
                xaxis2 = dict(scaleanchor = "x", title=xlabel),
                yaxis2 = dict(scaleanchor = "y"),
                )

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, shared_xaxes=True, subplot_titles=[plot1title, plot2title])
fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)
fig.update_layout(layout)
fig.show()

In [19]:
#compare 
beam_residual = beam_sky - beam_fp
phase_sky =  np.mod(np.angle(beam_sky), 2*np.pi)
phase_fp =  np.mod(np.angle(beam_fp), 2*np.pi)
############# plot the field #######################################
import plotly.graph_objects as go
y = P_rx_x
x = P_rx_z
# field = np.rot90(np.flip(field), 2, (0,1)) 
# field = np.rot90(np.flip(field), 3, (0,1)) 
phase = np.abs(phase_sky - phase_fp)
# phase = phase_sky/phase_fp
# phase = np.mod(phase+ 2.1+np.pi, np.pi*2)
amp = np.abs(beam_residual)/np.abs(beam_fp)
amp = np.where(amp<1, amp, 1)
amp = 20*np.log10(np.abs(beam_sky))-20*np.log10(np.abs(beam_fp))
# amp = np.where(amp<1, amp, 1)
# amp = 20*np.log10(np.abs(beam_residual))

title = 'Beam difference'
plot1title = "Phase [rad]"
plot2title = "Intensity [dB]"
xlabel = "x [mm]"
ylabel = "y [mm]"
plot_1 = go.Heatmap(
                    x=x, 
                    y=y,
                    z = phase,
                    colorscale='IceFire',
                    showscale=True, colorbar=dict(len=0.8, x=0.44),
                    showlegend = False,
                    # hoverinfo='name', 
                    name="phase"
                    )
plot_2 = go.Heatmap( 
                    x=x, 
                    y=y,
                    z = amp,
                    # z= np.abs(field_fp*field_fp),
                    colorscale='Jet',
                    showscale=True, 
                    colorbar=dict(len=0.8, x=1), 
                    showlegend = False,
                    # hoverinfo='name', 
                    name="power"
                    )
layout = go.Layout(title=title, autosize=True,
                width=1000, height=500, 
                margin=dict(l=50, r=50, b=65, t=90),
                xaxis1 = dict(title=xlabel),
                yaxis1 = dict(scaleanchor = 'x', title=ylabel),
                xaxis2 = dict(scaleanchor = "x", title=xlabel),
                yaxis2 = dict(scaleanchor = "y"),
                )

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, shared_xaxes=True, subplot_titles=[plot1title, plot2title])
fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)
fig.update_layout(layout)
fig.show()